In [1]:
# import libraries
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import random
import re
import os
# display options
pd.options.display.max_columns=999
pd.options.display.max_rows = 999

project_root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
file_path = os.path.join(project_root_dir, "data")
os.makedirs(file_path, exist_ok=True)

# function for loading data 
def load_data(filename, file_path=file_path):
    csv_path = os.path.join(file_path, filename)
    return pd.read_csv(csv_path)

# function for saving data as csv file
def save_dataframe(df, filename, file_path=file_path):
    """
    This function takes a dataframe and save it as a csv file.
    
    df: dataframe to save
    filename: Name to use for the csv file eg: 'my_file.csv'
    file_path = where to save the file
    """
    path = os.path.join(file_path, filename)
    df.to_csv(path, index=False)
    
def combine_all_years_data(function, year_list):
    """
    Common function for combining data for all the years for a 
    given table from ipl website or any other. All table have
    different functions to get the data from the websites.
    """
    try:
        # create an empty list to hold all the dataframes
        df_list = []
        # loop through each year and extract the data
        for year in year_list:
            # call the function to get the data for that year
            df = function(year)
            # append the data to the df list
            df_list.append(df)
            # add some random pause
            time.sleep(1 + 2*random.random())

        # concat all the dataframes
        df = pd.concat(df_list, ignore_index=True)
        
    except Exception as e:
        print(e)
        print(year)
    
    #return the dataframe
    return df 


# change the value for which you want to scrape the data defaults to 2008-2019
year_list = [year for year in range(2019,2007,-1)]   

# Maidens & Dots

### Dot Balls

In [2]:
url = "https://www.iplt20.com/stats/2019/most-dot-balls"
response = requests.get(url)
dots_html = response.text
dots_soup = bs(dots_html)
dots_table_data = dots_soup.find(class_ = "js-table")
# get the column names
col_names = []
for header in dots_table_data.find_all('th'):
    col_names.append(header.text.strip())
    
a_list = []
for data in dots_table_data.find_all('td'):
    a_list.append(' '.join(data.text.split()))

n = 13
final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
df = pd.DataFrame(final)
df.columns = col_names
df.head()

POS             PLAYER Mat Inns    Ov Runs Wkts Dots    Avg  Econ     SR 4w  \
0   1      Deepak Chahar  17   17  64.3  482   22  190  21.90  7.47  17.59  0   
1   2     Jasprit Bumrah  16   16  61.4  409   19  169  21.52  6.63  19.47  0   
2   3  Bhuvneshwar Kumar  15   15    59  461   13  168  35.46  7.81  27.23  0   
3   4        Rashid Khan  15   15    60  377   17  166  22.17  6.28  21.17  0   
4   5        Imran Tahir  17   17  64.2  431   26  149  16.57  6.69  14.84  2   

  5w  
0  0  
1  0  
2  0  
3  0  
4  0

In [3]:
# select only player name and Dots data
df = df[['PLAYER','Dots']]
# convert data type
df['Dots'] = pd.to_numeric(df['Dots'], errors='coerce').fillna(0)

In [4]:
def get_dot_balls_data(year):
    """This function gets the dot balls data for a particular year."""
    url = "https://www.iplt20.com/stats/{}/most-dot-balls".format(year)
    response = requests.get(url)
    dots_html = response.text
    dots_soup = bs(dots_html)
    dots_table_data = dots_soup.find(class_ = "js-table")
    # get the column names
    col_names = []
    for header in dots_table_data.find_all('th'):
        col_names.append(header.text.strip())

    a_list = []
    for data in dots_table_data.find_all('td'):
        a_list.append(' '.join(data.text.split()))

    n = 13
    final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
    df = pd.DataFrame(final)
    df.columns = col_names
    
    # select only player name and Dots data
    df = df[['PLAYER','Dots']]
    # convert data type
    df['Dots'] = pd.to_numeric(df['Dots'], errors='coerce').fillna(0)
    return df
    

In [5]:
dot_balls_2019 = get_dot_balls_data(2019)

In [6]:
dot_balls_2019.head()

PLAYER  Dots
0      Deepak Chahar   190
1     Jasprit Bumrah   169
2  Bhuvneshwar Kumar   168
3        Rashid Khan   166
4        Imran Tahir   149

### Maidens

In [7]:
url = "https://www.iplt20.com/stats/2019/most-maidens"
response = requests.get(url)
maidens_html = response.text
maidens_soup = bs(maidens_html)
maidens_table_data = maidens_soup.find(class_ = "js-table")
maidens_table_data

<div class="js-table">
<table class="table table--scroll-on-tablet top-players">
<tr class="top-players__header" data-widget="scroll-boundary">
<th class="top-players__freeze" title="Position">POS</th>
<th class="top-players__player top-players__freeze">PLAYER</th>
<th class="top-players__m top-players__padded" title="Matches">
                                Mat
                            </th>
<th class="top-players__inns" title="Innings">
                                Inns
                            </th>
<th class="top-players__ov" title="Overs">
                                Ov
                            </th>
<th class="top-players__r" title="Runs">
                                Runs
                            </th>
<th class="top-players__w" title="Wickets">
                                Wkts
                            </th>
<th class="top-players__maid" title="Maiden overs">
                                Maid
                            </th>
<th class="top-playe

In [8]:
# get the column names
col_names = []
for header in maidens_table_data.find_all('th'):
    col_names.append(header.text.strip())
col_names

['POS',
 'PLAYER',
 'Mat',
 'Inns',
 'Ov',
 'Runs',
 'Wkts',
 'Maid',
 'Avg',
 'Econ',
 'SR',
 '4w',
 '5w']

In [9]:
a_list = []
for data in maidens_table_data.find_all('td'):
    a_list.append(' '.join(data.text.split()))

n = 13
final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
df = pd.DataFrame(final)
df.columns = col_names
df.head()

POS           PLAYER Mat Inns    Ov Runs Wkts Maid    Avg  Econ     SR 4w 5w
0   1     Jofra Archer  11   11    43  291   11    2  26.45  6.76  23.45  0  0
1   2    Deepak Chahar  17   17  64.3  482   22    2  21.90  7.47  17.59  0  0
2   3      Rashid Khan  15   15    60  377   17    1  22.17  6.28  21.17  0  0
3   4  Ravindra Jadeja  16   16    54  343   15    1  22.86  6.35  21.60  0  0
4   5      Imran Tahir  17   17  64.2  431   26    1  16.57  6.69  14.84  2  0

In [10]:
# select only player name and maid column
df = df[['PLAYER','Maid']]
df['Maid'] = pd.to_numeric(df['Maid'], errors='coerce').fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PLAYER  18 non-null     object
 1   Maid    18 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


In [11]:
def get_maidens_data(year):
    """This function gets the player name and maidens
    data for a particular year.
    """
    try:
        url = "https://www.iplt20.com/stats/{}/most-maidens".format(year)
        response = requests.get(url)
        maidens_html = response.text
        maidens_soup = bs(maidens_html)
        maidens_table_data = maidens_soup.find(class_ = "js-table")
        # get the column names
        col_names = []
        for header in maidens_table_data.find_all('th'):
            col_names.append(header.text.strip())

        a_list = []
        for data in maidens_table_data.find_all('td'):
            a_list.append(' '.join(data.text.split()))

        n = 13
        final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
        df = pd.DataFrame(final)
        df.columns = col_names

        # select only player name and maid column
        df = df[['PLAYER','Maid']]
        # change data type
        df['Maid'] = pd.to_numeric(df['Maid'], errors='coerce').fillna(0)
        
    except Exception as e:
        print(e)
        print(year)
    
    return df
       

In [12]:
maidens_2019 = get_maidens_data(2019)

In [13]:
maidens_2019.head()

PLAYER  Maid
0     Jofra Archer     2
1    Deepak Chahar     2
2      Rashid Khan     1
3  Ravindra Jadeja     1
4      Imran Tahir     1

In [14]:
def get_dots_maidens(year):
    """
    Combine the dots, maidens and data into a single df.
    """
    try:
        dots_df = get_dot_balls_data(year)
        maidens_df = get_maidens_data(year)
        #hats_df = get_hat_tricks_data(year)

        df = pd.merge(left=dots_df, right=maidens_df,how='left', on=['PLAYER'])
        #df = pd.merge(left=df, right=hats_df,how='left',on=['PLAYER'])
        # fill missing values
        df.fillna(0, inplace=True)
    except Exception as e:
        print(e)
        print(year)
    
    return df

In [15]:
test_df = get_dots_maidens(2019)

In [16]:
test_df.head()

PLAYER  Dots  Maid
0      Deepak Chahar   190   2.0
1     Jasprit Bumrah   169   1.0
2  Bhuvneshwar Kumar   168   1.0
3        Rashid Khan   166   1.0
4        Imran Tahir   149   1.0

# Most Wickets

In [17]:
url = "https://www.iplt20.com/stats/2019/most-wickets"
response = requests.get(url)
bowling_html = response.text
bowling_soup = bs(bowling_html)
bowling_soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="IPLT20.com - Indian Premier League Official Website" name="twitter:title"/>
<meta content="ipl, iplt20, indian premier league, ipl cricket, ipl match, ipl live, ipl score, ipl scorecard, ipl stats, ipl schedule, ipl results, ipl points table, ipl teams, ipl videos, ipl teams, ipl news, BCCI IPL" name="keywords"/>
<meta content="website" property="og:type"/>
<meta content="Visit IPLT20.com the official IPLT20 website for minute-to-minute LIVE updates." name="description"/>
<meta content="Visit IPLT20.com the official IPLT20 website for minute-to-minute LIVE updates." name="twitter:description"/>
<meta content="IPLT20.com - Indian Premier League Official Website" property="og:title"/>
<title>IPLT20.com - Indian Premier League Official Website</title>
<meta content="Visit IPLT20.com the official IPLT20 website for minute-to-minute LIVE updates." property="og:description"/>
<meta charset="utf-8"/>
<meta content="width=device-width, ini

In [18]:
# get the table data
bowling_table_data = bowling_soup.find(class_ = "js-table")
bowling_table_data

<div class="js-table">
<table class="table table--scroll-on-tablet top-players">
<tr class="top-players__header" data-widget="scroll-boundary">
<th class="top-players__freeze" title="Position">POS</th>
<th class="top-players__player top-players__freeze">PLAYER</th>
<th class="top-players__m top-players__padded" title="Matches">
                                Mat
                            </th>
<th class="top-players__inns" title="Innings">
                                Inns
                            </th>
<th class="top-players__ov" title="Overs">
                                Ov
                            </th>
<th class="top-players__r" title="Runs">
                                Runs
                            </th>
<th class="top-players__w" title="Wickets">
                                Wkts
                            </th>
<th class="top-players__bbi" title="Bowling Figures">
                                BBI
                            </th>
<th class="top-play

In [19]:
# get the column names
col_names = []
for header in bowling_table_data.find_all('th'):
    col_names.append(header.text.strip())
col_names

['POS',
 'PLAYER',
 'Mat',
 'Inns',
 'Ov',
 'Runs',
 'Wkts',
 'BBI',
 'Avg',
 'Econ',
 'SR',
 '4w',
 '5w']

In [20]:
for data in bowling_table_data.find_all('td')[:20]:
    print(' '.join(data.text.split()))

1
Imran Tahir
17
17
64.2
431
26
0
16.57
6.69
14.84
2
0
2
Kagiso Rabada
12
12
47
368
25


In [21]:
a_list = []
for data in bowling_table_data.find_all('td'):
    a_list.append(' '.join(data.text.split()))

n = 13
final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
df = pd.DataFrame(final)
df.columns = col_names
df.head()

POS          PLAYER Mat Inns    Ov Runs Wkts BBI    Avg  Econ     SR 4w 5w
0   1     Imran Tahir  17   17  64.2  431   26   0  16.57  6.69  14.84  2  0
1   2   Kagiso Rabada  12   12    47  368   25   0  14.72  7.82  11.28  2  0
2   3   Deepak Chahar  17   17  64.3  482   22   0  21.90  7.47  17.59  0  0
3   4   Shreyas Gopal  14   14    48  347   20   0  17.35  7.22  14.40  0  0
4   5  Jasprit Bumrah  16   16  61.4  409   19   0  21.52  6.63  19.47  0  0

In [22]:
# Add the nationality of each player in the dataframe
nationality_list = []
for index, data in enumerate(bowling_table_data.find_all('tr')[1:]):
    try:
        nationality_list.append(data['data-nationality'])
    except Exception as e:
        print(e)
        print(index)
        # add none 
        nationality_list.append(None)
df['Nationality'] = nationality_list

In [23]:
df.sample(10)

POS              PLAYER Mat Inns    Ov Runs Wkts BBI    Avg   Econ     SR  \
99  100    Liam Livingstone   4    1     1   13    0   0      -  13.00      -   
37   38      Hardus Viljoen   6    6    23  222    7   0  31.71   9.65  19.71   
29   30      Jaydev Unadkat  11   11  37.2  398   10   0  39.80  10.66  22.40   
59   60          Andrew Tye   6    6    22  233    3   0  77.66  10.59  44.00   
62   63  Jagadeesha Suchith   1    1     4   28    2   0  14.00   7.00  12.00   
5     6       Khaleel Ahmed   9    9  34.5  287   19   0  15.10   8.23  11.00   
78   79  Kulwant Khejroliya   2    2     5   47    1   0  47.00   9.40  30.00   
70   71     Lockie Ferguson   5    5    17  183    2   0  91.50  10.76  51.00   
0     1         Imran Tahir  17   17  64.2  431   26   0  16.57   6.69  14.84   
83   84         Karn Sharma   1    1   2.5   33    1   0  33.00  11.64  17.00   

   4w 5w Nationality  
99  0  0    Overseas  
37  0  0    Overseas  
29  0  0      Indian  
59  0  0    Overseas  
62  0  0      Indian  
5   0  0      Indian  
78  0  0      Indian  
70  0  0    Overseas  
0   2  0    Overseas  
83  0  0      Indian

In [24]:
# Add the player link for more info in the dataframe
base_url = "https://www.iplt20.com"
player_link_list = []

# get all the links and add it to the list
for data in bowling_table_data.find_all('a'):
    player_link_list.append(base_url + data['href'])

# create a column with None value
df[14] = None
# iterate through each row and create a player name pattern
for index, row in df.iterrows():
    player_name = row['PLAYER'].replace(' ','-')
    player_regex = re.compile(r"{}".format(player_name),re.IGNORECASE)
    for item in player_link_list:
        # if the pattern matches any links
        if player_regex.search(item) != None:
            # then append it to that row of the df
            df.iloc[index,14] = item
# rename the column            
df.rename(columns={14:'Player Link'}, inplace=True)


# extract the player team name from the link and add to the df
team_regex = r"teams/(\w+-\w+-?\w+)"
df['Team'] = df['Player Link'].str.extract(team_regex, flags=re.IGNORECASE)
df['Team'] = df['Team'].apply(lambda x : str(x).title().replace('-',' '))

In [25]:
# convert data types from string to numeric
df['POS'] = pd.to_numeric(df['POS'], errors='coerce').fillna(0)
df['Mat'] = pd.to_numeric(df['Mat'], errors='coerce').fillna(0)
df['Inns'] = pd.to_numeric(df['Inns'], errors='coerce').fillna(0)
df['Ov'] = pd.to_numeric(df['Ov'], errors='coerce').fillna(0)
df['Runs'] = pd.to_numeric(df['Runs'], errors='coerce').fillna(0)
df['Wkts'] = pd.to_numeric(df['Wkts'], errors='coerce').fillna(0)
df['BBI'] = pd.to_numeric(df['BBI'], errors='coerce').fillna(0)
df['Avg'] = pd.to_numeric(df['Avg'], errors='coerce').fillna(0)
df['Econ'] = pd.to_numeric(df['Econ'], errors='coerce').fillna(0)
df['SR'] = pd.to_numeric(df['SR'], errors='coerce').fillna(0)
df['4w'] = pd.to_numeric(df['4w'], errors='coerce').fillna(0)
df['5w'] = pd.to_numeric(df['5w'], errors='coerce').fillna(0)

In [26]:
df.head()

POS          PLAYER  Mat  Inns    Ov  Runs  Wkts  BBI    Avg  Econ     SR  \
0    1     Imran Tahir   17    17  64.2   431    26    0  16.57  6.69  14.84   
1    2   Kagiso Rabada   12    12  47.0   368    25    0  14.72  7.82  11.28   
2    3   Deepak Chahar   17    17  64.3   482    22    0  21.90  7.47  17.59   
3    4   Shreyas Gopal   14    14  48.0   347    20    0  17.35  7.22  14.40   
4    5  Jasprit Bumrah   16    16  61.4   409    19    0  21.52  6.63  19.47   

   4w  5w Nationality                                        Player Link  \
0   2   0    Overseas  https://www.iplt20.com/teams/chennai-super-kin...   
1   2   0    Overseas  https://www.iplt20.com/teams/delhi-capitals/sq...   
2   0   0      Indian  https://www.iplt20.com/teams/chennai-super-kin...   
3   0   0      Indian  https://www.iplt20.com/teams/rajasthan-royals/...   
4   0   0      Indian  https://www.iplt20.com/teams/mumbai-indians/sq...   

                  Team  
0  Chennai Super Kings  
1       Delhi Capitals  
2  Chennai Super Kings  
3     Rajasthan Royals  
4       Mumbai Indians

In [27]:
def get_bowling_data(year):
    try:
        url = "https://www.iplt20.com/stats/{}/most-wickets".format(year)
        response = requests.get(url)
        bowling_html = response.text
        bowling_soup = bs(bowling_html)
        
        # get the table data
        bowling_table_data = bowling_soup.find(class_ = "js-table")
        
        # get the column names
        col_names = []
        for header in bowling_table_data.find_all('th'):
            col_names.append(header.text.strip())
            
        a_list = []
        for data in bowling_table_data.find_all('td'):
            a_list.append(' '.join(data.text.split()))

        n = 13
        final = [a_list[i:i + n] for i in range(0, len(a_list), n)]
        df = pd.DataFrame(final)
        df.columns = col_names
        
        # Add the nationality of each player in the dataframe
        nationality_list = []
        for index, data in enumerate(bowling_table_data.find_all('tr')[1:]):
            try:
                nationality_list.append(data['data-nationality'])
            except Exception as e:
                print(e)
                print(index)
                # add none 
                nationality_list.append(None)
        df['Nationality'] = nationality_list
        
        
        # Add the player link for more info in the dataframe
        base_url = "https://www.iplt20.com"
        player_link_list = []

        # get all the links and add it to the list
        for data in bowling_table_data.find_all('a'):
            player_link_list.append(base_url + data['href'])

        # create a column with None value
        df[14] = None
        # iterate through each row and create a player name pattern
        for index, row in df.iterrows():
            player_name = row['PLAYER'].replace(' ','-')
            player_regex = re.compile(r"{}".format(player_name),re.IGNORECASE)
            for item in player_link_list:
                # if the pattern matches any links
                if player_regex.search(item) != None:
                    # then append it to that row of the df
                    df.iloc[index,14] = item
        # rename the column            
        df.rename(columns={14:'Player Link'}, inplace=True)


        # extract the player team name from the link and add to the df
        team_regex = r"teams/(\w+-\w+-?\w+)"
        df['Team'] = df['Player Link'].str.extract(team_regex, flags=re.IGNORECASE)
        df['Team'] = df['Team'].apply(lambda x : str(x).title().replace('-',' '))
        
        # convert data types from string to numeric
        df['POS'] = pd.to_numeric(df['POS'], errors='coerce').fillna(0)
        df['Mat'] = pd.to_numeric(df['Mat'], errors='coerce').fillna(0)
        df['Inns'] = pd.to_numeric(df['Inns'], errors='coerce').fillna(0)
        df['Ov'] = pd.to_numeric(df['Ov'], errors='coerce').fillna(0)
        df['Runs'] = pd.to_numeric(df['Runs'], errors='coerce').fillna(0)
        df['Wkts'] = pd.to_numeric(df['Wkts'], errors='coerce').fillna(0)
        df['BBI'] = pd.to_numeric(df['BBI'], errors='coerce').fillna(0)
        df['Avg'] = pd.to_numeric(df['Avg'], errors='coerce').fillna(0)
        df['Econ'] = pd.to_numeric(df['Econ'], errors='coerce').fillna(0)
        df['SR'] = pd.to_numeric(df['SR'], errors='coerce').fillna(0)
        df['4w'] = pd.to_numeric(df['4w'], errors='coerce').fillna(0)
        df['5w'] = pd.to_numeric(df['5w'], errors='coerce').fillna(0)
        
        # extract the dots balls and maidens data
        df2 = get_dots_maidens(year) 
        
        # combine both the dataframes
        df = pd.merge(left=df, right=df2,how='left',on=['PLAYER'])
        # fill missing values
        df.fillna(0, inplace=True)
        
        # add season year
        df['Season'] = year
        
        
    except Exception as e:
        print(e)
        print(year)
        
        
        
    # return dataframe
    return df   
    

In [28]:
bowling_2019_df = get_bowling_data(2019)

In [29]:
bowling_2019_df.head()

POS          PLAYER  Mat  Inns    Ov  Runs  Wkts  BBI    Avg  Econ     SR  \
0    1     Imran Tahir   17    17  64.2   431    26    0  16.57  6.69  14.84   
1    2   Kagiso Rabada   12    12  47.0   368    25    0  14.72  7.82  11.28   
2    3   Deepak Chahar   17    17  64.3   482    22    0  21.90  7.47  17.59   
3    4   Shreyas Gopal   14    14  48.0   347    20    0  17.35  7.22  14.40   
4    5  Jasprit Bumrah   16    16  61.4   409    19    0  21.52  6.63  19.47   

   4w  5w Nationality                                        Player Link  \
0   2   0    Overseas  https://www.iplt20.com/teams/chennai-super-kin...   
1   2   0    Overseas  https://www.iplt20.com/teams/delhi-capitals/sq...   
2   0   0      Indian  https://www.iplt20.com/teams/chennai-super-kin...   
3   0   0      Indian  https://www.iplt20.com/teams/rajasthan-royals/...   
4   0   0      Indian  https://www.iplt20.com/teams/mumbai-indians/sq...   

                  Team   Dots  Maid  Season  
0  Chennai Super Kings  149.0   1.0    2019  
1       Delhi Capitals  113.0   0.0    2019  
2  Chennai Super Kings  190.0   2.0    2019  
3     Rajasthan Royals  107.0   1.0    2019  
4       Mumbai Indians  169.0   1.0    2019

In [30]:
# bowling_df = combine_all_years_data(get_bowling_data, year_list)

In [31]:
# bowling_df.shape

(1192, 19)

In [32]:
# bowling_df.sample(10)

POS                PLAYER  Mat  Inns    Ov  Runs  Wkts  BBI    Avg  \
848    55             Brett Lee   12    12  45.2   338     5    0  67.60   
1036   41          Sudeep Tyagi    8     8  19.0   136     5    0  27.20   
122    23           Lungi Ngidi    7     7  26.0   156    11    0  14.18   
504    11          Pravin Tambe   13    13  49.0   356    15    0  23.73   
602     9        Lasith Malinga   17    17  65.2   468    20    0  23.40   
846    53  Roelof van der Merwe    4     4  14.5    92     5    0  18.40   
999     6           Munaf Patel   11    11  34.5   241    16    0  15.06   
695     2          Sunil Narine   15    15  59.1   324    24    0  13.50   
33     34        Shardul Thakur   10     9  30.0   281     8    0  35.12   
77     78          Mohit Sharma    1     1   3.0    27     1    0  27.00   

      Econ     SR  4w  5w Nationality  \
848   7.45  54.40   0   0    Overseas   
1036  7.15  22.80   0   0      Indian   
122   6.00  14.18   1   0    Overseas   
504   7.26  19.60   1   0      Indian   
602   7.16  19.60   0   0    Overseas   
846   6.20  17.80   0   0    Overseas   
999   6.91  13.06   0   0      Indian   
695   5.47  14.79   1   1    Overseas   
33    9.36  22.50   0   0      Indian   
77    9.00  18.00   0   0      Indian   

                                            Player Link  \
848   https://www.iplt20.com/teams/kolkata-knight-ri...   
1036  https://www.iplt20.com/teams/chennai-super-kin...   
122   https://www.iplt20.com/teams/chennai-super-kin...   
504   https://www.iplt20.com/teams/rajasthan-royals/...   
602   https://www.iplt20.com/teams/mumbai-indians/sq...   
846   https://www.iplt20.com/teams/delhi-capitals/sq...   
999   https://www.iplt20.com/teams/rajasthan-royals/...   
695   https://www.iplt20.com/teams/kolkata-knight-ri...   
33    https://www.iplt20.com/teams/chennai-super-kin...   
77    https://www.iplt20.com/teams/chennai-super-kin...   

                       Team   Dots  Maid  Season  
848   Kolkata Knight Riders  108.0   1.0    2011  
1036    Chennai Super Kings   59.0   0.0    2009  
122     Chennai Super Kings   79.0   2.0    2018  
504        Rajasthan Royals   89.0   0.0    2014  
602          Mumbai Indians  147.0   0.0    2013  
846          Delhi Capitals   36.0   0.0    2011  
999        Rajasthan Royals  108.0   1.0    2009  
695   Kolkata Knight Riders  164.0   1.0    2012  
33      Chennai Super Kings   65.0   0.0    2019  
77      Chennai Super Kings    8.0   0.0    2019

In [33]:
# save_dataframe(bowling_df, 'bowling.csv', file_path)